# Stock Prediction

## Data preparation

In [1]:
# Imports the pandas library — a powerful Python tool for working with structured data (like tables, spreadsheets, and time series)
import pandas as pd

# Import math module for mathematical functions and constants, including those defined by the C standard.
import math

# Import NumPy for adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
import numpy as np

# Import MinMaxScaler for data preprocessing
from sklearn.preprocessing import MinMaxScaler

# Import the data from the csv file
data = pd.read_csv('Uber.csv')

# Reset the index
data = data.reset_index()

# Remove the rows with missing 'Close' values using boolean masking
data = data[data['Close'].notna()]

# Extract the Close Price column as a Data Frame
data_frame = pd.DataFrame()
data_frame['Close'] = data['Close']

# Convert the Data frame into a numpy array
dataset = data_frame.values

# Calculate the number of rows to train the model on.
training_data_len = math.ceil(len(dataset) - 50)

# Scale all of the data to be between 0 and 1
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = min_max_scaler.fit_transform(dataset)


## Extract the training data

In [2]:

# Extract the training data from the scaled dataset
training_data = scaled_data[0 : training_data_len , :]

# Split the data into x_train and y_train dataset
x_train = []
y_train = []

for i in range(10, len(training_data)):
    # Append the previous 10 values to x_train (10-day look back)
    x_train.append(training_data[i-10:i,0])
    
    # Append the current value to y_train
    y_train.append(training_data[i,0])

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Convert the data into 3D array for LSTM
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

## Extract the testing data

In [3]:

# Extract the testing data from the scaled dataset
testing_data = scaled_data[training_data_len - 10: , : ]

# Split the data into x_test and y_test dataset
x_test = []
y_test = []

for i in range(10, len(testing_data)):
    # Append the previous 10 values to x_test (10-day look back)
    x_test.append(testing_data[i-10:i,0])
    
    # Append the current value to y_test
    y_test.append(testing_data[i,0])

# Convert x_test and y_test to numpy arrays
x_test, y_test = np.array(x_test), np.array(y_test)

# Convert the data into 3D array for LSTM
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Models

In [4]:
# Import ML model dependencies
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, Conv1D, MaxPooling1D, Flatten
from keras_self_attention import SeqSelfAttention
from keras.optimizers import Adam
from keras.callbacks import CSVLogger
import time

## LSTM (50 Units)

### Build the LSTM model

In [5]:
# Build the LSTM model
model_lstm = Sequential([
    InputLayer(shape=(x_train.shape[1],1)),
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

# Compile the model
model_lstm.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')


### Initialize the CSV logger callback

In [6]:
csv_logger_lstm = CSVLogger("model_history_log_lstm.csv", append=False)

### Train the LSTM (50 units)

In [7]:

# Get the training start time
lstm_training_start_time = time.time()
 
#Train the model
model_lstm.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_lstm]) 

# Get the training end time
lstm_training_end_time = time.time()

# Calculate the total training time
lstm_total_training_time = lstm_training_end_time - lstm_training_start_time

# Print the total LSTMtm training time
print(lstm_total_training_time)


Epoch 1/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0233 - val_loss: 0.0084
Epoch 2/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0077 - val_loss: 0.0025
Epoch 3/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0066 - val_loss: 0.0021
Epoch 4/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0051 - val_loss: 0.0029
Epoch 5/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - val_loss: 0.0020
Epoch 6/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - val_loss: 0.0035
Epoch 7/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0047 - val_loss: 0.0021
Epoch 8/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - val_loss: 0.0018
Epoch 9/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0041 - val_loss: 0.0018
Epoch 10/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - val_loss: 0.0017
Epoch 11/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0041 - val_loss: 0.0034
Epoch 12/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0044 - val_lo

### Perform Prediction

In [8]:
# Predict the price values
lstm_prediction = model_lstm.predict(x_test) 

# Undo min max scaling
lstm_prediction = min_max_scaler.inverse_transform(lstm_prediction)

# Get the target values
lstm_prediction_target_values = min_max_scaler.inverse_transform(np.reshape(y_test, (50, 1)))

# Calculate of RMSE
rmse_lstm = np.sqrt(np.mean(((lstm_prediction - lstm_prediction_target_values)**2)))

# Print the RMSE
print(rmse_lstm)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
0.02769079293228206


## LSTM (1 Unit)

### Build the LSTM model

In [9]:
# Build the LSTM model
model_lstm_single_unit = Sequential([
    InputLayer(shape=(x_train.shape[1],1)),
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

# Compile the model
model_lstm_single_unit.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

### Initialize the CSV logger callback

In [10]:
csv_logger_lstm = CSVLogger("model_history_log_lstm_single_unit.csv", append=False)

### Train the LSTM (1 unit)

In [11]:

# Get the training start time
lstm_single_unit_training_start_time = time.time()
 
#Train the model
model_lstm_single_unit.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_lstm]) 

# Get the training end time
lstm_single_unit_training_end_time = time.time()

# Calculate the total training time
lstm_single_unit_total_training_time = lstm_single_unit_training_end_time - lstm_single_unit_training_start_time

# Print the total LSTMtm training time
print(lstm_single_unit_total_training_time)


Epoch 1/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0402 - val_loss: 0.0031
Epoch 2/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0088 - val_loss: 0.0026
Epoch 3/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0071 - val_loss: 0.0023
Epoch 4/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0062 - val_loss: 0.0024
Epoch 5/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0055 - val_loss: 0.0031
Epoch 6/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 - val_loss: 0.0019
Epoch 7/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 - val_loss: 0.0023
Epoch 8/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 - val_loss: 0.0050
Epoch 9/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049 - val_loss: 0.0025
Epoch 10/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0045 - val_loss: 0.0018
Epoch 11/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0047 - val_loss: 0.0018
Epoch 12/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0041 - val_lo

### Perform Prediction

In [12]:
# Predict the price values
lstm_single_unit_prediction = model_lstm.predict(x_test) 

# Undo min max scaling
lstm_single_unit_prediction = min_max_scaler.inverse_transform(lstm_single_unit_prediction)

# Get the target values
lstm_single_unit_prediction_target_values = min_max_scaler.inverse_transform(np.reshape(y_test, (50, 1)))

# Calculate of RMSE
rmse_lstm_single_unit = np.sqrt(np.mean(((lstm_single_unit_prediction - lstm_single_unit_prediction_target_values)**2)))

# Print the RMSE
print(rmse_lstm_single_unit)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
0.02769079293228206


## LSTM + Attention + CNN + LSTM

In [13]:
model_lstm_attention_cnn_lstm = Sequential([
    InputLayer(shape=(10, 1)),
    LSTM(units=50, return_sequences=True),
    SeqSelfAttention(attention_activation="relu"),
    Conv1D(filters=50, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    LSTM(units=50, return_sequences=False),
    Flatten(),
    Dense(units=1)
])
model_lstm_attention_cnn_lstm.summary()

model_lstm_attention_cnn_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 10, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ seq_self_attention              │ (None, 10, 50)         │         3,265 │
│ (SeqSelfAttention)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 10, 50)         │         7,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 5, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,466 (161.98 KB)

 Trainable params: 41,466 (161.98 KB)

 Non-trainable params: 0 (0.00 B)

### Initialize the CSV logger

In [14]:
csv_logger_lstm_attention_cnn_lstm = CSVLogger("model_history_log_lstm_attention_cnn_lstm.csv", append=False)

### Train the model

In [16]:

# Get the training start time
lstm_attention_cnn_lstm_start_time = time.time()
 
# Train the model
model_lstm_attention_cnn_lstm.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_lstm_attention_cnn_lstm]) 

# Get the training end time
lstm_attention_cnn_lstm_training_end_time = time.time()

# Calculate the total training time
lstm_attention_cnn_lstm_total_training_time= lstm_attention_cnn_lstm_training_end_time - lstm_attention_cnn_lstm_start_time

# Print the total training time
print(lstm_attention_cnn_lstm_total_training_time)

Epoch 1/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0725 - val_loss: 0.0060
Epoch 2/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0165 - val_loss: 0.0063
Epoch 3/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0153 - val_loss: 0.0089
Epoch 4/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0144 - val_loss: 0.0059
Epoch 5/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0140 - val_loss: 0.0062
Epoch 6/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0142 - val_loss: 0.0065
Epoch 7/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0141 - val_loss: 0.0053
Epoch 8/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0131 - val_loss: 0.0091
Epoch 9/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0137 - val_loss: 0.0057
Epoch 10/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0125 - val_loss: 0.0088
Epoch 11/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0110 - val_loss: 0.0102
Epoch 12/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0116 - val_lo

## LSTM + Attention + CNN + BiLSTM

In [17]:
model_lstm_attention_cnn_bilstm = Sequential([
    InputLayer(shape=(10, 1)),
    LSTM(units=50, return_sequences=True),
    SeqSelfAttention(attention_activation="relu"),
    Conv1D(filters=50, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(units=50, return_sequences=False)),
    Dense(units=1)
])

# Compile them model
model_lstm_attention_cnn_bilstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

### Initialize the CSV logger

In [18]:
csv_logger_lstm_attention_cnn_bilstm = CSVLogger("model_history_log_lstm_attention_cnn_bilstm.csv", append=False)

### Train the model

In [20]:

# Get the training start time
lstm_attention_cnn_bilstm_start_time = time.time()
 
#Train the model
model_lstm_attention_cnn_bilstm.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_lstm_attention_cnn_bilstm]) 

# Get the training end time
lstm_attention_cnn_bilstm_training_end_time = time.time()

# Calculate the total training time
lstm_attention_cnn_bilstm_total_training_time= lstm_attention_cnn_bilstm_training_end_time - lstm_attention_cnn_bilstm_start_time

# Print the total training time
print(lstm_attention_cnn_bilstm_total_training_time)

Epoch 1/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0597 - val_loss: 0.0067
Epoch 2/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0171 - val_loss: 0.0076
Epoch 3/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0154 - val_loss: 0.0062
Epoch 4/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0145 - val_loss: 0.0082
Epoch 5/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0136 - val_loss: 0.0091
Epoch 6/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0134 - val_loss: 0.0055
Epoch 7/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0124 - val_loss: 0.0054
Epoch 8/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0128 - val_loss: 0.0085
Epoch 9/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0123 - val_loss: 0.0058
Epoch 10/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0120 - val_loss: 0.0070
Epoch 11/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0098 - val_loss: 0.0042
Epoch 12/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0096 - val_l

## LSTM + Attention + LSTM

### Build the model

In [21]:
model_lstm_attention_lstm = Sequential([
    InputLayer(shape=(10, 1)),
    LSTM(units=50, return_sequences=True),
    SeqSelfAttention(attention_activation="relu"),
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

# Compile them model
model_lstm_attention_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

### Initialize the CSV logger

In [22]:
csv_logger_lstm_attention_lstm = CSVLogger("model_history_log_lstm_attention_lstm.csv", append=False)

### Train the model

In [24]:

# Get the training start time
lstm_attention_lstm_training_start_time = time.time()
 
# Train the model
model_lstm_attention_lstm.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_lstm_attention_lstm]) 

# Get the training end time
lstm_attention_lstm_training_end_time = time.time()

# Calculate the total training time
lstm_attention_lstm_total_training_time= lstm_attention_lstm_training_end_time - lstm_attention_lstm_training_start_time

# Print the total training time
print(lstm_attention_lstm_total_training_time)

Epoch 1/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0660 - val_loss: 0.0071
Epoch 2/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0188 - val_loss: 0.0051
Epoch 3/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0185 - val_loss: 0.0118
Epoch 4/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0173 - val_loss: 0.0052
Epoch 5/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0163 - val_loss: 0.0111
Epoch 6/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0170 - val_loss: 0.0057
Epoch 7/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0154 - val_loss: 0.0064
Epoch 8/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0151 - val_loss: 0.0094
Epoch 9/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0164 - val_loss: 0.0054
Epoch 10/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0145 - val_loss: 0.0109
Epoch 11/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0137 - val_loss: 0.0050
Epoch 12/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0136 - val_l

## LSTM + Attention + BiLSTM

### Build the model

In [25]:
model_lstm_attention_bilstm = Sequential([
    InputLayer(shape=(10, 1)),
    LSTM(units=50, return_sequences=True),
    SeqSelfAttention(attention_activation="relu"),
    Bidirectional(LSTM(units=50, return_sequences=False)),
    Dense(units=1)
])

# Compile them model
model_lstm_attention_bilstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

### Initialize the CSV logger

In [26]:
csv_logger_lstm_attention_bilstm = CSVLogger("model_history_log_lstm_attention_bilstm.csv", append=False)

### Train the model

In [28]:
# Get the training start time
lstm_attention_bilstm_start_time = time.time()
 
#Train the model
model_lstm_attention_bilstm.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_lstm_attention_bilstm]) 

# Get the training end time
lstm_attention_bilstm_training_end_time = time.time()

# Calculate the total training time
lstm_attention_bilstm_total_training_time= lstm_attention_bilstm_training_end_time - lstm_attention_bilstm_start_time

# Print the total training time
print(lstm_attention_bilstm_total_training_time)

Epoch 1/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0649 - val_loss: 0.0055
Epoch 2/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0170 - val_loss: 0.0063
Epoch 3/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0152 - val_loss: 0.0074
Epoch 4/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0146 - val_loss: 0.0109
Epoch 5/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0154 - val_loss: 0.0100
Epoch 6/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0160 - val_loss: 0.0104
Epoch 7/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0146 - val_loss: 0.0064
Epoch 8/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0128 - val_loss: 0.0057
Epoch 9/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0128 - val_loss: 0.0055
Epoch 10/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0122 - val_loss: 0.0043
Epoch 11/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0119 - val_loss: 0.0073
Epoch 12/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0102 - val_l

## CNN + LSTM + Attention + LSTM

### Build the model

In [29]:
model_cnn_lstm_attention_lstm = Sequential([
    InputLayer(shape=(10, 1)),
    Conv1D(filters=50, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    SeqSelfAttention(attention_activation="relu"),
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

# Compile them model
model_cnn_lstm_attention_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

### Initialize the CSV logger

In [30]:
csv_logger_cnn_lstm_attention_lstm = CSVLogger("model_history_log_cnn_lstm_attention_lstm.csv", append=False)

### Train the model

In [32]:
# Get the training start time
cnn_lstm_attention_lstm_start_time = time.time()
 
#Train the model
model_cnn_lstm_attention_lstm.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_cnn_lstm_attention_lstm]) 

# Get the training end time
cnn_lstm_attention_lstm_training_end_time = time.time()

# Calculate the total training time
cnn_lstm_attention_lstm_total_training_time= cnn_lstm_attention_lstm_training_end_time - cnn_lstm_attention_lstm_start_time

# Print the total training time
print(cnn_lstm_attention_lstm_total_training_time)

Epoch 1/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.1047 - val_loss: 0.0048
Epoch 2/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0164 - val_loss: 0.0040
Epoch 3/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0127 - val_loss: 0.0062
Epoch 4/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0119 - val_loss: 0.0064
Epoch 5/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0111 - val_loss: 0.0066
Epoch 6/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0109 - val_loss: 0.0071
Epoch 7/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0110 - val_loss: 0.0070
Epoch 8/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0098 - val_loss: 0.0044
Epoch 9/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0094 - val_loss: 0.0063
Epoch 10/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0090 - val_loss: 0.0042
Epoch 11/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0101 - val_loss: 0.0030
Epoch 12/40
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0083 - val_lo

## BiLSTM + Attention + CNN + BiLSTM

In [33]:
model_bilstm_attention_cnn_bilstm = Sequential([
    InputLayer(shape=(10, 1)),
    Bidirectional(LSTM(units=50, return_sequences=True)),
    SeqSelfAttention(attention_activation="relu"),
    Conv1D(filters=50, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(units=50, return_sequences=False)),
    Dense(units=1)
])

# Compile them model
model_bilstm_attention_cnn_bilstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

### Initialize the CSV Logger

In [34]:
csv_logger_bilstm_attention_cnn_bilstm = CSVLogger("model_history_log_bilstm_attention_cnn_bilstm.csv", append=False)

### Train the model

In [ ]:
# Get the training start time
bilstm_attention_cnn_bilstm_start_time = time.time()
 
#Train the model
model_bilstm_attention_cnn_bilstm.fit(x_train, y_train, batch_size=10, epochs=40,validation_data=(x_test, scaled_data[training_data_len : , : ]),callbacks=[csv_logger_bilstm_attention_cnn_bilstm]) 

# Get the training end time
bilstm_attention_cnn_bilstm_end_time = time.time()

# Calculate the total training time
bilstm_attention_cnn_bilstm_total_training_time= bilstm_attention_cnn_bilstm_end_time - bilstm_attention_cnn_bilstm_start_time

# Print the total training time
print(bilstm_attention_cnn_bilstm_total_training_time)

NameError: name 'csv_logger_cnn_lstm_attention' is not defined